# 🏦 Credit Card Application Review with Azure AI Agents Streaming

## Overview

This notebook demonstrates **Azure AI Agents** with **streaming** for a credit card application review workflow. Watch real-time analysis as two specialized agents process an application.

### 💼 Industry Use Case: Credit Card Application Pipeline

A customer submits a credit card application:
1. **Credit Analyst Agent** - Reviews application and assesses creditworthiness
2. **Underwriter Agent** - Makes final decision with terms and conditions (streaming output)

### ⚠️ Important Financial Disclaimer
> **This notebook is for educational purposes only.** The credit decision logic is simplified and should not be used for actual lending decisions. Always follow regulatory requirements and consult compliance teams.

### Key Concepts

| Concept | Description |
|---------|-------------|
| **Azure AI Agent Service** | Using Azure-hosted agents with the Agent Framework |
| **Streaming** | Real-time token generation via `AgentRunUpdateEvent` |
| **register_agent()** | Lazy agent creation with factory functions |
| **WorkflowBuilder** | Pipeline construction with edges between agents |

### Architecture

```
Credit Card Application
    ↓
Credit Analyst Agent (creditworthiness assessment)
    ↓ streaming
Underwriter Agent (approval decision + terms)
    ↓ streaming
Final Credit Decision
```

## Prerequisites

- ✅ Azure AI Agent Service configured
- ✅ Environment variable: `AZURE_AI_PROJECT_CONNECTION_STRING`
- ✅ Azure CLI authentication: Run `az login`

## 1️⃣ Import Libraries and Load Environment

In [ ]:
from agent_framework import AgentRunUpdateEvent, ChatAgent, WorkflowBuilder, WorkflowOutputEvent
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from dotenv import load_dotenv

load_dotenv('../../.env')
print("✅ Libraries imported and environment loaded")

## 2️⃣ Define Agent Factory Functions

The factory function pattern ensures proper resource management:

1. **Lazy Initialization**: Agents created only when client is ready
2. **Shared Session**: All agents use the same authenticated client
3. **Clean Separation**: Agent config separate from workflow logic

### 📊 Credit Analyst Agent
- Reviews application financials
- Assesses debt-to-income ratio
- Evaluates credit history factors

### ✅ Underwriter Agent  
- Makes approval/denial decision
- Sets credit limit and APR
- Provides terms and conditions

In [ ]:
def create_credit_analyst(client: AzureAIAgentClient) -> ChatAgent:
    """Creates a Credit Analyst agent for application review."""
    return client.as_agent(
        name="CreditAnalyst",
        instructions=(
            "You are a Credit Analyst at a retail bank reviewing credit card applications.\n"
            "For each application:\n"
            "1. Evaluate the applicant's income and employment stability\n"
            "2. Assess debt-to-income ratio\n"
            "3. Review credit score and history factors\n"
            "4. Identify any risk factors or concerns\n"
            "5. Provide a preliminary creditworthiness assessment\n"
            "Be thorough but concise in your analysis."
        ),
    )


def create_underwriter(client: AzureAIAgentClient) -> ChatAgent:
    """Creates an Underwriter agent for final credit decisions."""
    return client.as_agent(
        name="Underwriter",
        instructions=(
            "You are a Credit Card Underwriter at a retail bank.\n"
            "Based on the Credit Analyst's assessment, make a final decision:\n"
            "1. APPROVE, DECLINE, or CONDITIONAL APPROVAL\n"
            "2. If approved, recommend credit limit ($500-$15,000 range)\n"
            "3. If approved, recommend APR range based on risk\n"
            "4. List any conditions for conditional approvals\n"
            "5. Include standard regulatory disclosures\n"
            "Provide a clear, structured decision."
        ),
    )

## 3️⃣ Build and Run the Credit Application Workflow

### Workflow Configuration
- **`register_agent()`**: Lazy initialization with lambdas
- **`set_start_executor()`**: Credit Analyst is the entry point
- **`add_edge()`**: Flow from analyst → underwriter
- **`output_response=True`**: Underwriter's decision is captured

### Streaming Events
- **`AgentRunUpdateEvent`**: Real-time chunks as agents analyze
- **`WorkflowOutputEvent`**: Final credit decision

In [ ]:
async def main() -> None:
    async with AzureCliCredential() as cred, AzureAIAgentClient(credential=cred) as client:
        # Build the credit card application workflow
        workflow = (
            WorkflowBuilder()
            .register_agent(lambda: create_credit_analyst(client), name="credit_analyst")
            .register_agent(lambda: create_underwriter(client), name="underwriter", output_response=True)
            .set_start_executor("credit_analyst")
            .add_edge("credit_analyst", "underwriter")
            .build()
        )

        # Sample credit card application
        credit_application = """
        CREDIT CARD APPLICATION
        =======================
        Applicant: Sarah Johnson
        Application Date: 2024-01-15
        Product: Rewards Visa Card
        
        PERSONAL INFORMATION:
        - Age: 32
        - Residence: Homeowner (5 years)
        
        FINANCIAL INFORMATION:
        - Annual Income: $85,000
        - Employment: Marketing Manager at Tech Corp (4 years)
        - Monthly Rent/Mortgage: $1,800
        - Existing Credit Cards: 2 (total limit $12,000, utilization 25%)
        - Auto Loan Balance: $15,000 (monthly payment $350)
        - Credit Score: 745
        
        REQUESTED CREDIT LIMIT: $10,000
        """

        print("💳 CREDIT CARD APPLICATION REVIEW")
        print("=" * 60)
        print(credit_application)
        print("=" * 60 + "\n")

        last_executor_id: str | None = None

        events = workflow.run_stream(credit_application)
        async for event in events:
            if isinstance(event, AgentRunUpdateEvent):
                eid = event.executor_id
                if eid != last_executor_id:
                    if last_executor_id is not None:
                        print()
                    agent_emoji = "📊" if "analyst" in eid else "✅"
                    print(f"\n{agent_emoji} {eid.upper()}:", end=" ", flush=True)
                    last_executor_id = eid
                print(event.data, end="", flush=True)
            elif isinstance(event, WorkflowOutputEvent):
                print("\n\n" + "=" * 60)
                print("📋 FINAL CREDIT DECISION")
                print("=" * 60)
                print(event.data)

        print("\n" + "=" * 60)
        print("✅ Credit card application review complete!")
        print("\n⚠️ DISCLAIMER: This is a demonstration only. Actual credit")
        print("   decisions require full underwriting and regulatory compliance.")


await main()

## 📝 Key Takeaways

### Azure AI Agents for Credit Decisioning

| Concept | Description |
|---------|-------------|
| **Factory Pattern** | `lambda: create_agent(client)` for lazy initialization |
| **register_agent()** | Registers factories instead of pre-created agents |
| **Streaming Events** | `AgentRunUpdateEvent` for real-time progress |
| **Async Context** | `async with` ensures proper resource cleanup |

### FSI Benefits of Streaming Workflows

| Benefit | Application |
|---------|-------------|
| **Transparency** | Show reasoning process to compliance teams |
| **Audit Trail** | Record decision-making flow |
| **Early Detection** | Catch issues before final decision |
| **Customer Experience** | Faster perceived response times |

### Next Steps
- Add **Credit Bureau Tool** for real credit data
- Implement **Human-in-the-Loop** for edge cases
- Add **Compliance Executor** for required disclosures
- Integrate **Fraud Detection** agent